# Sentiment Analysis

Table of contents
* [Setup](#intro)
* [1. Using a sentiment lexicon](#approach1) (basic approach)
* [2. Using existing NLP tools](#approach2) (classical methods)
* [3. Using pre-trained transformer-based models](#approach3) (good results)
* [4. ChatGPT-like models](#approach4) (excellent results)

## Setup

In [ ]:
%pip install textblob
%pip install vaderSentiment
%pip install -q transformers

In [ ]:
from textblob import TextBlob
import pandas as pd
import nltk
if 'google.colab' in str(get_ipython()):
    nltk.download('punkt')

<p id="approach1"></p>

## Approach 1: Applying a sentiment lexicon (basic approach)

Let's start by reading a sentiment lexicon

In [ ]:
lexname="https://raw.githubusercontent.com/fmmb/Text-Mining/main/data/NRC-lexicon.csv"
data = pd.read_csv(lexname, encoding="utf-8", index_col=["English"])

In [ ]:
print(f"Lexicon Size: {len(data)} words")
display(data.sample(5))

We will be using the dictionary `lex` instead of the dataframe, since the performance is much better that way

In [ ]:
lex = (data['Positive']-data['Negative']).to_dict()

Let's test some well-known words

In [ ]:
for w in ['good', 'ok', 'dislike', 'bad']:
    print(w, "=>", lex.get(w, 0))

Now, to calculate the sentiment of a sentence, we only have to sum the polarity of each individual word, and check the final result.

In [ ]:
text = 'This is a lovely and beautiful place , but I hate the corner'
polarity = 0
for w in text.split():
    polarity += lex.get(w, 0)
    print(w, lex.get(w, 0) )
print("Final score:" , polarity)

Even better with a function ...

In [ ]:
def sentiment(text):
    score = 0
    for w in text.split():
        score += lex.get(w, 0)
    if score >= 0:
        return "POS"
    else:
        return "NEG"

In [ ]:
text = 'This is a lovely and beautiful place , but I hate the corner'
sentiment(text)

We could now process some texts with our classifier

In [ ]:
texts = [ "The movie was good.",
          "I hate the movie",
          "The movie was not good.",
          "I really think this product sucks.",
          "Really great product.",
          "I don't like this product"]
for text in texts:
    print(text, "=>", sentiment(text))

<p id="approach2"></p>

## Approach 2: Using existing NLP tools
There are many existing tools for sentiment analysis, such as `TextBlob` and `sentiment Vader`, amongst others. This section presents examples on how to use these two tools

### TextBlob
`TextBlob` will be used to perform our initial sentiment analysis tests.
Please consider the following list of texts, stored in the variable `texts` 

In [ ]:
texts = [ "The movie was good.",
          "I hate the movie",
          "The movie was not good.",
          "I really think this product sucks.",
          "Really great product.",
          "I don't like this product"]

We can now print each one of the individual texts, together with it's sentiment score. A value above 0 means that the sentiment is positive, while a value below 0, means it is negative.

In [ ]:
for t in texts:
    print(t, "==>", TextBlob(t).sentiment.polarity)

The previous code assumes that the text is already split into sentences, which may not be the case of texts comming from sources, such as *web pages* or *blogs*. However, your can give the whole text to `textblob`, and it is able to automatically split the texts into sentences, as follows.

In [ ]:
mytext = """The movie was good. The movie was not good. I really think this product sucks.
Really great product. I don't like this product."""
text=TextBlob(mytext)

In [ ]:
for s in text.sentences:
    print(s, "==> ", s.sentiment.polarity)

### Evaluation

Let's automatically classify a set of texts and evaluate the corresponding performance. 

In order to be able to do that, one has to provide the true labels together with the data. However, please note that the classification system won't be able to see the true labels. 

In [ ]:
texts = ["I love chocolate",
        "I hate to eat", 
        "I don't love anyone",
        "I like cakes",
        "I don't fail"]
tags=["pos", "neg", "neg", "pos", "pos"]

Let's do a small test with one of these examples...

In [ ]:
print("TEXT:", texts[0])
print("SENTIMENT:", TextBlob(texts[0]).sentiment.polarity)

In [ ]:
correct=0
total=0
for i in range(len(texts)):
    polarity = TextBlob(texts[i]).sentiment.polarity
    print(f"SCORE: {polarity:4}, TAG: {tags[i]}, TEXT: {texts[i]}")
    if polarity >=0 and tags[i] == "pos":
        correct +=1
    elif polarity <0 and tags[i] == "neg":
        correct += 1
    total += 1

print(f"correct: {correct}, wrong: {total-correct}, accuracy: {correct/total}")

### Vader Sentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sa = SentimentIntensityAnalyzer()

texts = [
    "This food is amazing and tasty !",
    "Exoplanets are planets outside the solar system",
    "It is sad to see such a bad behavior"
]

for text in texts:
    score = sa.polarity_scores(text)["compound"]
    print('TEXT:', text)
    print('  SENTIMENT:', score)

After computing the polarity score, we can also calculate the proportion of each sentiment in a sentence using the keys: "pos", "neu" and "neg" 

In [ ]:
for text in texts:
    scores = sa.polarity_scores(text)
    print('TEXT:', text)
    print('  SCORES:', scores)

<p id="approach3"></p>

## Approach 3: Using pre-trained transformer-based models

### The easy way ...

In [ ]:
from transformers import pipeline

In [ ]:
#sentiment_pipeline = pipeline("sentiment-analysis")
sentiment_pipeline = pipeline("text-classification")
#sentiment_pipeline = pipeline("sentiment-analysis", model="finiteautomata/bertweet-base-sentiment-analysis")

In [ ]:
data = ["I love you", "I hate you", "the unemployment is increasing"]

In [ ]:
sentiment_pipeline(data)

### Fine-tunning: improved models (optional)

Some practical readings...
* [Getting Started with Sentiment Analysis using Python](https://huggingface.co/blog/sentiment-analysis-python)
* [Sentiment Analysis in 10 Minutes with BERT and TensorFlow](https://towardsdatascience.com/sentiment-analysis-in-10-minutes-with-bert-and-hugging-face-294e8a04b671)

<p id="approach4"></p>

## Approach 4: ChatGPT-like models

Please use [ChatGPT](https://chat.openai.com/), [Gemini](https://gemini.google.com/) or a similar model to calculate the sentiment of the following sentences
* This food is amazing and tasty!
* Exoplanets are planets outside the solar system
* It is sad to see such a bad behavior

Try changing the prompt as follows:
> Calculate the sentiment of the following sentences. Produce the output in the following format: TEXT, SENTIMENT (Positive, Neutral, Negative)
            ...